In [3]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
import unicodedata
import mysql.connector
import re
import ipynb
from ipynb.fs.full.SqlQuery import brandAdd
from ipynb.fs.full.SqlQuery import categorieAdd
from ipynb.fs.full.SqlQuery import AddProduct
from ipynb.fs.full.SqlQuery import getCagId
from ipynb.fs.full.SqlQuery import getBrandId
from ipynb.fs.full.SqlQuery import AddProductAttribut
from ipynb.fs.full.SqlQuery import AddLocalStoreProduct
from ipynb.fs.full.SqlQuery import Addlocalstors

In [4]:
mydb = mysql.connector.connect(
  host="localhost",
  user="root",
  password="",
  database="scraping_data"
)

In [14]:
def itechStore_links():    
    itech_list=[]
    web = requests.get('https://www.itechstore.tn/')
    parse = BeautifulSoup(web.content, 'html.parser')
    itech_links= parse.find_all('a', class_="nav-link")
    i=0
    for it in itech_links:
        i=i+1
        if(i>1 and i<10):
            link=it['href']
            info=it.text.strip()
            tech_dict={
                'link':link,
                'info':info
            }
            itech_list.append(tech_dict)
    return itech_list

In [10]:
def tech_details(link):
    web = requests.get(link)
    parse = BeautifulSoup(web.content, 'html.parser')
    try:
        name=parse.find('h1', class_='h1 page-title').text
    except:
        name='Unkonwn'
    try:
        uid= parse.find('div', class_='product-reference').span.text
    except:
        uid='Undifined'
    try:
        stock= parse.find('div', class_='rte-content').p.span.text.replace('!','').strip()
    except:
        stock='non disonible'
    try:
        brand = parse.find('div', class_='product_header_container clearfix').meta['content']
    except:
        brand='Unknown'
    try:
        brand_logo = parse.find('div', class_='product_header_container clearfix').div.a.get('href')
    except:
        brand_logo = 'None'
    try:
        des = parse.find('div', class_='rte-content').text
        des= re.sub(r'(\s+|\n)', ' ', des)
    except:
        des='None'
    label, data='',''
    feautre= parse.find_all('dt', class_='name')
    value = parse.find_all('dd', class_='value')
    fich_list=[]
    for n in feautre:
        fich_tech={
            'label': n.text,
            'data': ''
            }
        fich_list.append(fich_tech)
    i=0
    for v in value:
        fich_list[i]['data']=v.text
        i +=1
    details={
          'name':name, 
          'uid':uid,
          'stock':stock,
          'brand':brand,
          'brand_logo':brand_logo,
          'des':des,
          'fich_list':fich_list
      }
    return details

In [15]:
print(tech_details('https://www.itechstore.tn/iphone-12-pro/1367-242-iphone-12-pro-128go-graphite-.html#/41-couleur-bleu_pacifique'))

{'name': 'iPhone 12 Pro Max 256Go - Bleu Pacifique', 'uid': 'MGDF3AA/A', 'stock': 'Disponible', 'brand': 'Apple', 'brand_logo': 'https://www.itechstore.tn/3_apple', 'des': 'Disponible ! Vers la vitesse et au-delà Une puce A14 Bionic, la plus rapide sur smartphone. Un écran Super Retina, OLED bord à bord. Le Ceramic Shield, qui multiplie par quatre la résistance aux chutes. Et le mode Nuit sur chaque appareil photo.', 'fich_list': [{'label': 'Ecran', 'data': 'OLED tout écran de 6,1 pouces (diagonale)\nÉcran Super Retina XDR'}, {'label': "Taille d'écran", 'data': '6,7 pouces'}, {'label': 'Résolution', 'data': '2 778 x 1 284 pixels à 458 ppp'}, {'label': 'Processeur', 'data': 'Neural Engine nouvelle génération\nPuce A14 Bionic'}, {'label': 'Stockage ', 'data': '256Go'}, {'label': 'Caméra frontale', 'data': 'Caméra TrueDepth 12 Mp\nMode Portrait avec effet bokeh avancé et Contrôle de la profondeur\nOuverture ƒ/2,2\nÉclairage de portrait avec six effets (Naturel, Studio, Contour, Scène, Scè

In [12]:
def next_url(url, x):
    if(x == 1):
        url=url+'?page='+str(x)
    else:
        url=url[:-1]
        if(x>10):
          url=url[:-1]
        url=url+str(x)
    web = requests.get(url)
    parse = BeautifulSoup(web.content, 'html.parser')
    if(parse.find('div', class_='alert alert-warning')):
        return False

    return url

In [17]:
def priceConv(price):
    c=''
    for p in price:
        if(p.isdigit()):
            c=c+p
    return int(c)

In [20]:
# mycursor = mydb.cursor()
supCatId=0
list= itechStore_links()
for l in list:
    link=l['link']
    info=l['info']
    categorieAdd(info, link, 20)
    supCatId=getCagId(info,20)
    web = requests.get(link)
    parse = BeautifulSoup(web.content, 'html.parser')
    for x in range(1, 10):
        print(link)
        link= next_url(link,x)
        print(link)
        if(not(link == False)):
            web = requests.get(link)
            parse = BeautifulSoup(web.content, 'html.parser')
            card = parse.find_all('article', class_='product-miniature product-miniature-default product-miniature-grid product-miniature-layout-1 js-product-miniature')
            for c in card:
                art_link= c.find('h3', class_='h3 product-title').a['href']
                try:
                    img= c.find('div', class_='thumbnail-container').a.img.get('data-src')
                except:
                    img='None'
                try:
                    price= c.find('span', class_='product-price').text
                    price=priceConv(price)
                except:
                    price=0
                det= tech_details(art_link)
                brandAdd(det['brand'], det['brand_logo'], 20)
                supBrandId=getBrandId(det['brand'], 20)
                prodId=AddProduct(det['name'], art_link, det['uid'], det['des'], price, det['stock'], 20, supBrandId, supCatId, img)
                if(det['fich_list']):
                    AddProductAttribut(det['fich_list'], prodId)
        else:
            break

Data Base Categorie Duplication Error

https://www.itechstore.tn/3-mac
https://www.itechstore.tn/3-mac?page=1
Product Duplicate Error
https://www.itechstore.tn/3-mac?page=1
https://www.itechstore.tn/3-mac?page=2
https://www.itechstore.tn/3-mac?page=2
https://www.itechstore.tn/3-mac?page=3
https://www.itechstore.tn/3-mac?page=3
False
https://www.itechstore.tn/4-ipad
https://www.itechstore.tn/4-ipad?page=1
Product Duplicate Error
Product Duplicate Error
Product Duplicate Error
https://www.itechstore.tn/4-ipad?page=1
https://www.itechstore.tn/4-ipad?page=2
Product Duplicate Error
https://www.itechstore.tn/4-ipad?page=2
False
https://www.itechstore.tn/5-iphone
https://www.itechstore.tn/5-iphone?page=1
https://www.itechstore.tn/5-iphone?page=1
https://www.itechstore.tn/5-iphone?page=2
https://www.itechstore.tn/5-iphone?page=2
https://www.itechstore.tn/5-iphone?page=3
Product Duplicate Error
Product Duplicate Error
Product Duplicate Error
Product Duplicate Error
Product Duplicate Error
https